In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Flatten, BatchNormalization, Conv1D, MaxPooling1D
from keras import Input
# from keras.layers import Conv1D, MaxPooling1D

from keras.metrics import mean_squared_error
import tensorflow as tf

from google.colab import drive
drive.mount('/content/drive')

with open('/content/drive/My Drive/ai_project/data/arrayed_img.pkl', 'rb') as f:
    df = pickle.load(f)

n = 100 # 100일간 데이터 input
test_num = 10 # 10일간 데이터 output

df_forplot = df[-test_num:]
df = df[:-test_num]

temp_data = df.values

scaler = MinMaxScaler()
temp_data_scaled = scaler.fit_transform(temp_data.reshape(-1, 1))[:, 0]

# 이후 10일 데이터 예측
X_scaled = np.array([temp_data_scaled[i:i+n] for i in range(len(df)-n-test_num+1)])
y_scaled = np.array([temp_data_scaled[i+n:i+n+test_num] for i in range(len(df)-n-test_num+1)])

X_scaled = X_scaled.reshape((-1, n, 1))
y_scaled = y_scaled.reshape((-1, 10, 1))

def plot_result(his):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    ax1.plot(his.history['loss'])
    ax1.plot(his.history['val_loss'])
    ax1.set_title('Model Loss')
    ax1.set_ylabel('Loss')
    ax1.set_xlabel('Epoch')
    ax1.legend(['Train', 'Validation'], loc='upper right')

    ax2.plot(his.history['mae'])
    ax2.plot(his.history['val_mae'])
    ax2.set_title('Model MAE')
    ax2.set_ylabel('MAE')
    ax2.set_xlabel('Epoch')
    ax2.legend(['Train', 'Validation'], loc='upper right')

    plt.show()

def make_pred(num, model):
    data_pred = np.array(df['value'].values[:-n])
    for i in range(num):
        temp = data_pred[-n:]
        pred = model.predict(temp.reshape(1, -1))
        data_pred = np.append(data_pred, pred)
    return data_pred[-num:]

from keras.callbacks import EarlyStopping

custom_early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled = train_test_split(np.array(X_scaled), np.array(y_scaled), test_size=0.2, random_state=42)

mse = tf.keras.losses.MeanSquaredError()
mae = tf.keras.losses.MeanAbsoluteError()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


KeyError: ignored

In [3]:
import pickle

from google.colab import drive
drive.mount('/content/drive')

with open('/content/drive/My Drive/ai_project/data/arrayed_img.pkl', 'rb') as f:
    df = pickle.load(f)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df = df[:100]

In [5]:
df.head()

,img_193_loc,img_304_loc
2010-09-01,"[[[0.0], [0.0], [0.0], [0.0], [1.0], [0.0], [0...","[[[0.0], [1.0], [0.0], [0.0], [0.0], [0.0], [0..."
2010-09-02,"[[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...","[[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0..."
2010-09-03,"[[[1.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...","[[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0..."
2010-09-04,"[[[1.0], [0.0], [1.0], [0.0], [0.0], [0.0], [0...","[[[0.0], [0.0], [0.0], [0.0], [1.0], [1.0], [1..."
2010-09-05,"[[[1.0], [0.0], [1.0], [1.0], [5.0], [1.0], [0...","[[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1..."


In [6]:
df.values

array([[array([[[0.],
                [0.],
                [0.],
                ...,
                [0.],
                [0.],
                [0.]],

               [[1.],
                [1.],
                [1.],
                ...,
                [0.],
                [0.],
                [1.]],

               [[2.],
                [0.],
                [1.],
                ...,
                [0.],
                [1.],
                [0.]],

               ...,

               [[0.],
                [0.],
                [0.],
                ...,
                [0.],
                [0.],
                [1.]],

               [[0.],
                [0.],
                [1.],
                ...,
                [0.],
                [0.],
                [0.]],

               [[0.],
                [1.],
                [1.],
                ...,
                [0.],
                [0.],
                [1.]]], dtype=float32), array([[[0.],
                   

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense, Concatenate
from tensorflow.keras.models import Model

# Parameters
image_shape = (512, 512, 1)  # Image dimensions
num_float_data = 1           # Number of float data points
output_length = 10           # Output length from each image processing CNN

# Function to create a CNN model for image processing
def create_image_cnn(input_shape, output_length, name):
    image_input = Input(shape=input_shape, name=f'{name}_input')
    x = Conv2D(32, (3, 3), activation='relu')(image_input)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = Flatten()(x)
    image_features = Dense(output_length, activation='relu', name=f'{name}_features')(x)
    return Model(inputs=image_input, outputs=image_features, name=f'{name}_cnn')

# 1. CNNs for Each Image
cnn1 = create_image_cnn(image_shape, output_length, 'image1')
cnn2 = create_image_cnn(image_shape, output_length, 'image2')

# 2. Inputs
image1_input = Input(shape=image_shape, name='image1_input')
image2_input = Input(shape=image_shape, name='image2_input')
float_input = Input(shape=(num_float_data,), name='float_input')

# Process each image
image1_features = cnn1(image1_input)
image2_features = cnn2(image2_input)

# Concatenation of Image Features and Float Data
concatenated = Concatenate()([image1_features, image2_features, float_input])

# 3. 1D CNN for Concatenated Data
x = Dense(64, activation='relu')(concatenated)
x = Dense(64, activation='relu')(x)
final_output = Dense(1, activation='linear', name='final_output')(x)  # Assuming a single float output

# Model
model = Model(inputs=[image1_input, image2_input, float_input], outputs=final_output)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Model Summary
model.summary()

# Train the model
# model.fit([image1_data, image2_data, float_data], target_values, epochs=10, batch_size=32)
